# Tải Dữ Liệu ĐBSCL từ Google Earth Engine

Script này giúp tải dữ liệu mưa (CHIRPS) và nhiệt độ (ERA5) cho 13 tỉnh Đồng Bằng Sông Cửu Long

## BƯỚC 1: CÀI ĐẶT & KẾT NỐI

In [3]:
# Cài đặt thư viện (nếu chưa có)
!pip install geemap h3 -q

import os
import ee
import geemap
import rasterio
import h3
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
from rasterio.mask import mask as rasterio_mask
from scipy.spatial import cKDTree

# Xac dinh thu muc project (uu tien thu muc chua folder data)
PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

# Thu muc luu du lieu khi chay local (co the override bang bien moi truong GEE_DATA_DIR)
LOCAL_DATA_DIR = Path(os.getenv("GEE_DATA_DIR", PROJECT_ROOT / "data" / "processed" / "gee_exports"))
LOCAL_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Thu muc du lieu local: {LOCAL_DATA_DIR}")
print("✅ Tat ca thu vien da import thanh cong")

✅ Thu muc du lieu local: d:\Mekong_DGGS\data\processed\gee_exports
✅ Tat ca thu vien da import thanh cong


In [4]:
# 1. Thiet lap thu muc luu file tren local (thay cho Google Drive)
print(" Luu file ra thu muc local, khong su dung Google Drive...")
out_dir = LOCAL_DATA_DIR
os.makedirs(out_dir, exist_ok=True)
print(f" Thu muc luu du lieu: {out_dir}")

 Luu file ra thu muc local, khong su dung Google Drive...
 Thu muc luu du lieu: d:\Mekong_DGGS\data\processed\gee_exports


In [ ]:
# 2. Kết nối Google Earth Engine với PROJECT ID
MY_PROJECT_ID = 'ringed-reach-480215-u2'  # <--- ID của bạn đã được điền sẵn

print(f"🔑 Đang đăng nhập GEE với Project: {MY_PROJECT_ID}...")
try:
    # Thử khởi tạo trực tiếp
    ee.Initialize(project=MY_PROJECT_ID)
except Exception as e:
    # Nếu chưa đăng nhập, kích hoạt cửa sổ đăng nhập
    ee.Authenticate()
    ee.Initialize(project=MY_PROJECT_ID)

print(" Kết nối GEE thành công!")

🔑 Đang đăng nhập GEE với Project: ringed-reach-480215-u2...
 Kết nối GEE thành công!


## BƯỚC 2: TẢI DỮ LIỆU ĐBSCL

In [ ]:
# Danh sách 13 tỉnh ĐBSCL
mekong_provinces = [
    'An Giang', 'Bac Lieu', 'Ben Tre', 'Ca Mau', 'Can Tho city',
    'Dong Thap', 'Hau Giang', 'Kien Giang', 'Long An',
    'Soc Trang', 'Tien Giang', 'Tra Vinh', 'Vinh Long'
]

print(f" Đang xử lý dữ liệu cho 13 tỉnh ĐBSCL...")

# Lấy ranh giới hành chính
vietnam = ee.FeatureCollection("FAO/GAUL/2015/level1")
roi = vietnam.filter(ee.Filter.inList('ADM1_NAME', mekong_provinces)).geometry()

# Thời gian: Tháng 9/2023 (Ví dụ mùa lũ)
START = '2023-09-01'
END = '2023-09-30'

### A. DỮ LIỆU MƯA (CHIRPS)

In [ ]:
print(" Đang tải dữ liệu Mưa (CHIRPS)...")
rain_img = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
    .filterDate(START, END) \
    .select('precipitation') \
    .sum() \
    .clip(roi)

# Xuất file Mưa ra Google Drive
geemap.ee_export_image(
    rain_img,
    filename=os.path.join(out_dir, 'Mua_DBSCL_Sep2023.tif'),
    scale=5000, # Độ phân giải ~5km
    region=roi,
    file_per_band=False
)

### B. DỮ LIỆU NHIỆT ĐỘ (ERA5)

In [ ]:
print(" Đang tải dữ liệu Nhiệt độ (ERA5)...")
# Lấy nhiệt độ trung bình tháng, đổi từ Kelvin sang Celsius
temp_img = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
    .filterDate(START, END) \
    .select('temperature_2m') \
    .first() \
    .clip(roi) \
    .subtract(273.15)

# Xuất file Nhiệt độ ra Google Drive
geemap.ee_export_image(
    temp_img,
    filename=os.path.join(out_dir, 'NhietDo_DBSCL_Sep2023.tif'),
    scale=10000, # Độ phân giải ~10km
    region=roi,
    file_per_band=False
)

## Hoàn tất

### C. DỮ LIỆU LAND COVER (ESA/WorldCover)

In [ ]:
print(" Đang tạo tác vụ xuất dữ liệu Land Cover (ESA/WorldCover v200 - 2021) sang Google Drive...")
landcover_img = ee.ImageCollection("ESA/WorldCover/v200") \
    .first() \
    .clip(roi)

# Khởi tạo tác vụ xuất file Land Cover ra Google Drive bằng ee.batch.Export.image.toDrive
task = ee.batch.Export.image.toDrive(
    image=landcover_img,
    description='LandCover_DBSCL_2021',
    folder=os.path.basename(out_dir), # Lấy tên thư mục 'Du_Lieu_DBSCL' từ out_dir
    fileNamePrefix='LandCover_DBSCL_2021',
    scale=30, # Độ phân giải 30m
    region=roi, # Sử dụng trực tiếp đối tượng roi
    maxPixels=1e13 # Tăng giới hạn pixel để tránh lỗi
)

task.start()

print(f"✅ Tác vụ xuất dữ liệu Land Cover đã được khởi tạo. Bạn có thể kiểm tra tiến độ trong tab \"Tasks\" của Google Earth Engine hoặc trong Google Drive: {os.path.join(out_dir, 'LandCover_DBSCL_2021.tif')}")

### D. DỮ LIỆU SÔNG (River - JRC Global Surface Water)

In [ ]:
print("Đang tạo tác vụ xuất dữ liệu Sông (JRC Global Surface Water - max_extent tổng hợp 1984-2020) sang Google Drive...")
water_img = ee.Image("JRC/GSW1_4/GlobalSurfaceWater") \
    .select('max_extent') \
    .clip(roi)
filename_river_prefix = 'River_Water_DBSCL_MaxExtent'
print("✅ Sẽ sử dụng dữ liệu max_extent (tổng hợp 1984-2020) cho dữ liệu sông.")

# Khởi tạo tác vụ xuất file Sông ra Google Drive bằng ee.batch.Export.image.toDrive
task_river = ee.batch.Export.image.toDrive(
    image=water_img,
    description=filename_river_prefix,
    folder=os.path.basename(out_dir),
    fileNamePrefix=filename_river_prefix,
    scale=30, # Độ phân giải ~30m (scale mặc định của YearlyHistory là 30m)
    region=roi,
    maxPixels=1e13
)

task_river.start()

print(f" Tác vụ xuất dữ liệu Sông đã được khởi tạo. Bạn có thể kiểm tra tiến độ trong tab \"Tasks\" của Google Earth Engine hoặc trong Google Drive: {os.path.join(out_dir, filename_river_prefix + '.tif')}")


### E. DỮ LIỆU DEM - CHỈ SỐ CAO ĐỘ (Digital Elevation Model)

In [ ]:
print(" Đang tạo tác vụ xuất dữ liệu DEM (USGS SRTM 1 Arc-Second Global - 30m) sang Google Drive...")
# Sử dụng USGS/SRTMGL1_003 (SRTM 1 Arc-Second Global - 30m)
dem_img = ee.Image("USGS/SRTMGL1_003") \
    .select('elevation') \
    .clip(roi)

# Khởi tạo tác vụ xuất file DEM ra Google Drive bằng ee.batch.Export.image.toDrive
task_dem = ee.batch.Export.image.toDrive(
    image=dem_img,
    description='DEM_DBSCL',
    folder=os.path.basename(out_dir),
    fileNamePrefix='DEM_DBSCL',
    scale=30, # Độ phân giải 30m
    region=roi,
    maxPixels=1e13
)

task_dem.start()

print(f" Tác vụ xuất dữ liệu DEM đã được khởi tạo. Bạn có thể kiểm tra tiến độ trong tab \"Tasks\" của Google Earth Engine hoặc trong Google Drive: {os.path.join(out_dir, 'DEM_DBSCL.tif')}")

# Tính Slope từ DEM
print(" Đang tạo tác vụ xuất dữ liệu Slope (từ DEM) sang Google Drive...")
slope_img = ee.Terrain.slope(dem_img)

# Khởi tạo tác vụ xuất file Slope ra Google Drive bằng ee.batch.Export.image.toDrive
task_slope = ee.batch.Export.image.toDrive(
    image=slope_img,
    description='Slope_DBSCL',
    folder=os.path.basename(out_dir),
    fileNamePrefix='Slope_DBSCL',
    scale=30, # Độ phân giải 30m (giống DEM)
    region=roi,
    maxPixels=1e13
)

task_slope.start()

print(f" Tác vụ xuất dữ liệu Slope đã được khởi tạo. Output: {os.path.join(out_dir, 'Slope_DBSCL.tif')}")


In [ ]:
print("-" * 50)
print(f"🎉 HOÀN TẤT! Hãy vào Google Drive mở thư mục 'Du_Lieu_DBSCL' để lấy file.")

## PHẦN 3: XỬ LÝ STATIC SPATIAL FEATURES (Landcover, River, DEM)

### 3.1 Tạo H3 Grid từ Ranh Giới ĐBSCL (Độc lập)

### 3.0 Làm sạch Ranh Giới ĐBSCL (Loại bỏ Đảo Nhỏ - theo clean_shapefile.py)

In [ ]:
# ============================================================
# HÀM CHUYỂN ĐỔI - GEE Geometry sang GeoDataFrame
# ============================================================

def gee_geometry_to_gdf(gee_geom):
    """Chuyển GEE Geometry sang GeoDataFrame (WGS84)"""
    geom_dict = ee.Geometry(gee_geom).getInfo()
    if geom_dict['type'] == 'Polygon':
        coords = geom_dict['coordinates'][0]
        return gpd.GeoDataFrame(
            geometry=[Polygon(coords)],
            crs="EPSG:4326"
        )
    else:
        # Multi-polygon case
        from shapely.geometry import MultiPolygon
        coords_list = geom_dict['coordinates']
        polys = [Polygon(coords[0]) for coords in coords_list]
        return gpd.GeoDataFrame(
            geometry=[MultiPolygon(polys)],
            crs="EPSG:4326"
        )

# ============================================================
# HÀM LÀMẠCH RANH GIỚI - Loại bỏ đảo nhỏ (theo clean_shapefile.py)
# ============================================================

def clean_roi_remove_small_islands(roi_gdf_input, min_area_km2=600):
    """
    Làm sạch ranh giới bằng cách loại bỏ các đảo/mảnh đất nhỏ

    Quy trình (theo clean_shapefile.py):
    1. Chuyển sang hệ tọa độ metric (UTM 48N) để tính diện tích chính xác
    2. Explode: Tách MultiPolygon thành các Polygon riêng lẻ
    3. Tính diện tích (km²)
    4. Lọc: Chỉ giữ mảnh đất > min_area_km2
    5. Dissolve: Gộp lại thành 1 geometry
    6. Chuyển về WGS84 (EPSG:4326)

    Args:
        roi_gdf_input: GeoDataFrame với ranh giới gốc (EPSG:4326)
        min_area_km2: Ngưỡng diện tích tối thiểu (km²) - Mặc định 600 km²

    Returns:
        GeoDataFrame sau khi làm sạch (1 polygon hợp nhất)
    """
    print(f"🧹 Làm sạch ranh giới ĐBSCL - Loại bỏ đảo/mảnh đất < {min_area_km2} km²...")

    # BƯỚC 1: Chuyển sang UTM 48N (EPSG:32648) để tính diện tích chính xác
    print(f"  ➜ BƯỚC 1: Chuyển sang hệ metric (EPSG:32648)...")
    gdf_metric = roi_gdf_input.to_crs("EPSG:32648")

    # BƯỚC 2: Explode - Tách MultiPolygon thành các Polygon riêng lẻ
    print(f"  ➜ BƯỚC 2: Explode - Tách các tỉnh (MultiPolygon) thành mảnh riêng...")
    gdf_exploded = gdf_metric.explode(index_parts=True).reset_index(drop=True)
    print(f"     Sau explode: {len(gdf_exploded)} mảnh đất riêng lẻ")

    # BƯỚC 3: Tính diện tích (km²)
    print(f"  ➜ BƯỚC 3: Tính diện tích (km²)...")
    gdf_exploded['area_km2'] = gdf_exploded.geometry.area / 1e6

    # BƯỚC 4: Lọc - Chỉ giữ mảnh đất > ngưỡng
    print(f"  ➜ BƯỚC 4: Lọc các mảnh > {min_area_km2} km²...")
    removed_count = len(gdf_exploded[gdf_exploded['area_km2'] <= min_area_km2])
    gdf_clean = gdf_exploded[gdf_exploded['area_km2'] > min_area_km2].copy()
    print(f"     Loại bỏ: {removed_count} mảnh nhỏ")
    print(f"     Giữ lại: {len(gdf_clean)} mảnh lớn")

    # Hiển thị thông tin chi tiết
    print(f"\n  📊 Thông tin mảnh đất được giữ lại:")
    for idx, (_, row) in enumerate(gdf_clean.iterrows(), 1):
        if 'ADM1_NAME' in gdf_clean.columns:
            print(f"     {idx}. {row.get('ADM1_NAME', 'N/A')}: {row['area_km2']:.1f} km²")
        else:
            print(f"     {idx}. Mảnh {idx}: {row['area_km2']:.1f} km²")

    # BƯỚC 5: Dissolve - Gộp lại thành 1 geometry duy nhất
    print(f"  ➜ BƯỚC 5: Dissolve - Gộp tất cả mảnh lại...")
    gdf_dissolved = gdf_clean.dissolve()

    # BƯỚC 6: Chuyển về WGS84 (EPSG:4326)
    print(f"  ➜ BƯỚC 6: Chuyển về WGS84 (EPSG:4326)...")
    gdf_final = gdf_dissolved.to_crs("EPSG:4326")

    print(f"✅ Làm sạch hoàn tất! Ranh giới sạch: 1 polygon hợp nhất (Đất liền + Đảo lớn)")
    return gdf_final

# CHẠY: Làm sạch ranh giới ĐBSCL
print("=" * 70)
print("PHẦN 3.0: LÀM SẠCH RANH GIỚI ĐBSCL (Loại bỏ Đảo Nhỏ)")
print("=" * 70)

# Lấy ranh giới 13 tỉnh từ GEE
mekong_provinces = [
    'An Giang', 'Bac Lieu', 'Ben Tre', 'Ca Mau', 'Can Tho city',
    'Dong Thap', 'Hau Giang', 'Kien Giang', 'Long An',
    'Soc Trang', 'Tien Giang', 'Tra Vinh', 'Vinh Long'
]

vietnam = ee.FeatureCollection("FAO/GAUL/2015/level1")
roi_dbscl = vietnam.filter(ee.Filter.inList('ADM1_NAME', mekong_provinces)).geometry()

# Chuyển GEE geometry sang GeoDataFrame
roi_gdf_raw = gee_geometry_to_gdf(roi_dbscl)

# Áp dụng làm sạch
MIN_AREA_KM2 = 600  # Ngưỡng: Loại bỏ các đảo < 600 km²
roi_gdf_clean = clean_roi_remove_small_islands(roi_gdf_raw, min_area_km2=MIN_AREA_KM2)

print(f"\n✅ Ranh giới sạch sẽ được dùng cho phần 3.1 (H3 Grid generation)")


PHẦN 3.0: LÀM SẠCH RANH GIỚI ĐBSCL (Loại bỏ Đảo Nhỏ)
🧹 Làm sạch ranh giới ĐBSCL - Loại bỏ đảo/mảnh đất < 600 km²...
  ➜ BƯỚC 1: Chuyển sang hệ metric (EPSG:32648)...
  ➜ BƯỚC 2: Explode - Tách các tỉnh (MultiPolygon) thành mảnh riêng...
     Sau explode: 33 mảnh đất riêng lẻ
  ➜ BƯỚC 3: Tính diện tích (km²)...
  ➜ BƯỚC 4: Lọc các mảnh > 600 km²...
     Loại bỏ: 20 mảnh nhỏ
     Giữ lại: 13 mảnh lớn

  📊 Thông tin mảnh đất được giữ lại:
     1. Mảnh 1: 3333.3 km²
     2. Mảnh 2: 2244.6 km²
     3. Mảnh 3: 1527.7 km²
     4. Mảnh 4: 2598.2 km²
     5. Mảnh 5: 5154.9 km²
     6. Mảnh 6: 1391.5 km²
     7. Mảnh 7: 1670.4 km²
     8. Mảnh 8: 3524.0 km²
     9. Mảnh 9: 2283.6 km²
     10. Mảnh 10: 3377.3 km²
     11. Mảnh 11: 5658.3 km²
     12. Mảnh 12: 4473.9 km²
     13. Mảnh 13: 2382.9 km²
  ➜ BƯỚC 5: Dissolve - Gộp tất cả mảnh lại...
  ➜ BƯỚC 6: Chuyển về WGS84 (EPSG:4326)...
✅ Làm sạch hoàn tất! Ranh giới sạch: 1 polygon hợp nhất (Đất liền + Đảo lớn)

✅ Ranh giới sạch sẽ được dùng cho 

In [ ]:
# ============================================================
# PHẦN 3.1: SETUP - Cấu hình
# ============================================================

# ============================================================
# PHẦN 3.1: HÀM H3 - Định nghĩa hàm hỗ trợ (Compatibility v3/v4)
# ============================================================

# Hàm lấy tên function H3 tùy phiên bản
def get_h3_func_names():
    v = int(h3.__version__.split('.')[0])
    if v >= 4:
        return h3.latlng_to_cell, h3.cell_to_boundary
    else:
        return h3.geo_to_h3, h3.h3_to_geo_boundary

to_cell_func, to_boundary_func = get_h3_func_names()

# ============================================================
# PHẦN 3.1: RANH GIỚI - SỬ DỤNG RANH GIỚI ĐÃ LÀM SẠCH (từ 3.0)
# ============================================================

# Nếu 3.0 đã chạy, sử dụng roi_gdf_clean
# Nếu chưa chạy, tạo ranh giới thô (không làm sạch)
if 'roi_gdf_clean' in locals() and not roi_gdf_clean.empty:
    roi_gdf = roi_gdf_clean
    print(f"📂 Sử dụng ranh giới ĐÃ LÀM SẠCH từ phần 3.0 (loại bỏ đảo < 600 km²)")
else:
    print(f"⚠️  Phần 3.0 chưa chạy. Lấy ranh giới thô từ GEE (bao gồm cả đảo nhỏ)")
    mekong_provinces = [
        'An Giang', 'Bac Lieu', 'Ben Tre', 'Ca Mau', 'Can Tho city',
        'Dong Thap', 'Hau Giang', 'Kien Giang', 'Long An',
        'Soc Trang', 'Tien Giang', 'Tra Vinh', 'Vinh Long'
    ]
    vietnam = ee.FeatureCollection("FAO/GAUL/2015/level1")
    roi_dbscl = vietnam.filter(ee.Filter.inList('ADM1_NAME', mekong_provinces)).geometry()
    roi_gdf = gee_geometry_to_gdf(roi_dbscl)

# Cấu hình đường dẫn dữ liệu tĩnh (sử dụng v2)
data_folder = '/content/drive/MyDrive/Du_Lieu_DBSCL_v2'

landcover_path = os.path.join(data_folder, 'LandCover_DBSCL_2021.tif')
river_path = os.path.join(data_folder, 'River_Water_DBSCL_MaxExtent.tif')
dem_path = os.path.join(data_folder, 'DEM_DBSCL.tif')
slope_path = os.path.join(data_folder, 'Slope_DBSCL.tif')
salinity_path = os.path.join(data_folder, 'DoMan_DBSCL_2023.tif')

print(f"📂 Data folder: {data_folder}")
print("🔍 Ranh giới ĐBSCL được tải thành công")
roi_gdf_dbscl = roi_gdf  # Đặt tên để phân biệt
print(f"✅ Ranh giới chuẩn bị cho H3 grid generation")

RES = 7  # Resolution 7 ~ 1.2km
print(f"\n🔷 Tạo lưới H3 (Resolution={RES})...")

# ============================================================
# BẢNG TRA CỨU CHIỀU CẠNH H3 (DỰA TRÊN BẢNG H3 CHÍNH THỨC)
# ============================================================
H3_EDGE_LEN_METERS = {
    5: 8544.4, 6: 3229.6, 7: 1220.6,
    8: 461.4, 9: 174.4, 10: 65.9
}
edge_len = H3_EDGE_LEN_METERS.get(RES, 2000)
BUFFER_DIST = edge_len * 2.0  # Hệ số an toàn: 2x chiều dài cạnh

def generate_h3_grid_from_polygon(gdf, resolution):
    """
    Tạo lưới H3 từ polygon bằng chiến lược: Buffer -> Polyfill -> Intersect

    Nguyên lý:
    - BƯỚC 1: Nới rộng ranh giới (Buffer) để bắt các ô ở mép biển
    - BƯỚC 2: Polyfill trên ranh giới đã nới rộng
    - BƯỚC 3: Lọc lại để chỉ giữ ô có giao cắt với ranh giới gốc (Intersect)

    Lợi ích: Không mất dữ liệu ở vùng biên giới, cửa sông, ven biển
    """

    # BƯỚC 1: BUFFER - Nối rộng ranh giới
    print(f"--> BƯỚC 1: Buffer {BUFFER_DIST}m...")
    gdf_metric = gdf.to_crs('EPSG:32648')  # Chuyển sang UTM để buffer chính xác
    gdf_buffered = gdf_metric.copy()
    gdf_buffered['geometry'] = gdf_metric.buffer(BUFFER_DIST)
    gdf_buffered_ll = gdf_buffered.to_crs('EPSG:4326')  # Chuyển lại WGS84 cho H3

    # BƯỚC 2: POLYFILL - Sinh H3 trên vùng buffer
    print(f"--> BƯỚC 2: Polyfill (trên vùng đã nối rộng)...")
    hex_set = set()

    for _, row in gdf_buffered_ll.iterrows():
        geom = row.geometry
        # Xử lý cả Polygon và MultiPolygon
        try:
            geoms_list = geom.geoms if hasattr(geom, 'geoms') else [geom]

            for single_geom in geoms_list:
                if hasattr(h3, 'polygon_to_cells'):  # H3 v4
                    # Sử dụng LatLngPoly
                    from h3 import LatLngPoly
                    outer = [(lat, lon) for lon, lat in single_geom.exterior.coords]
                    holes = []
                    for interior in single_geom.interiors:
                        holes.append([(lat, lon) for lon, lat in interior.coords])
                    h3_poly = LatLngPoly(outer, holes)
                    hexes = h3.polygon_to_cells(h3_poly, resolution)
                else:  # H3 v3
                    # Sử dụng polyfill
                    from shapely.geometry import mapping
                    geojson = mapping(single_geom)
                    hexes = h3.polyfill(geojson, resolution, geo_json_conformant=True)
                hex_set.update(hexes)
        except Exception as e:
            print(f"   ⚠️ Lỗi polyfill: {e}")
            continue

    print(f"--> Sinh ra được: {len(hex_set)} ô (bao gồm cả ô thừa ngoài biên gốc)")

    # BƯỚC 3: VALIDATE (INTERSECT) - Lọc giao cắt với ranh giới gốc
    print(f"--> BƯỚC 3: Validate (lọc ô có giao cắt với ranh giới gốc)...")

    valid_hexes = []
    union_poly = unary_union(gdf.geometry.values)  # Gộp ranh giới gốc

    for h_idx in hex_set:
        # Tái tạo polygon lục giác
        try:
            boundary = to_boundary_func(h_idx)
            if hasattr(h3, 'cell_to_boundary'):  # H3 v4 trả về (lat, lon)
                poly_coords = [(lon, lat) for lat, lon in boundary]
            else:  # H3 v3 trả về (lon, lat)
                poly_coords = boundary
            poly_geom = Polygon(poly_coords)

            # Chỉ lấy nếu ô H3 chạm/giao cắt với ranh giới gốc
            if poly_geom.intersects(union_poly):
                valid_hexes.append(h_idx)
        except Exception as e:
            print(f"   ⚠️ Lỗi boundary: {e}")
            continue

    print(f"✅ Sau lọc Intersect: {len(valid_hexes)} ô H3 (loại bỏ ô thừa ngoài biên)")

    # Tái tạo geometry cho các ô hợp lệ
    hex_polys = []
    for h_idx in valid_hexes:
        try:
            boundary = to_boundary_func(h_idx)
            if hasattr(h3, 'cell_to_boundary'):  # H3 v4
                poly_coords = [(lon, lat) for lat, lon in boundary]
            else:  # H3 v3
                poly_coords = boundary
            hex_polys.append(Polygon(poly_coords))
        except:
            continue

    h3_result = gpd.GeoDataFrame(
        {'h3_index': valid_hexes[:len(hex_polys)]},
        geometry=hex_polys,
        crs="EPSG:4326"
    )

    print(f"📊 Mức hợp lý H3: {len(h3_result)} ô (mục tiêu: ~7.000–8.000)")
    return h3_result

# Tạo H3 grid toàn ĐBSCL (độc lập, không phụ thuộc rain/temp)
h3_grid = generate_h3_grid_from_polygon(roi_gdf, RES)
print(f"📊 H3 grid ĐBSCL: {len(h3_grid)} ô")
print(f"✅ Cấu hình hoàn tất")


📂 Sử dụng ranh giới ĐÃ LÀM SẠCH từ phần 3.0 (loại bỏ đảo < 600 km²)
📂 Data folder: /content/drive/MyDrive/Du_Lieu_DBSCL_v2
🔍 Ranh giới ĐBSCL được tải thành công
✅ Ranh giới chuẩn bị cho H3 grid generation

🔷 Tạo lưới H3 (Resolution=7)...
--> BƯỚC 1: Buffer 2441.2m...
--> BƯỚC 2: Polyfill (trên vùng đã nối rộng)...
--> Sinh ra được: 7144 ô (bao gồm cả ô thừa ngoài biên gốc)
--> BƯỚC 3: Validate (lọc ô có giao cắt với ranh giới gốc)...
✅ Sau lọc Intersect: 6959 ô H3 (loại bỏ ô thừa ngoài biên)
📊 Mức hợp lý H3: 6959 ô (mục tiêu: ~7.000–8.000)
📊 H3 grid ĐBSCL: 6959 ô
✅ Cấu hình hoàn tất


### 3.2 Các hàm xử lý (Polygon Masking)

In [ ]:
# ============================================================
# VISUALIZATION: Quy trình 3 Bước Buffer -> Polyfill -> Validate
# ============================================================
if 'h3_grid' in locals() and not h3_grid.empty and 'roi_gdf' in locals():
    print("🎨 Vẽ biểu đồ quá trình H3 grid generation...")

    # Lấy lại vùng buffer (để vẽ)
    gdf_metric = roi_gdf.to_crs('EPSG:32648')
    gdf_buffered = gdf_metric.copy()
    gdf_buffered['geometry'] = gdf_metric.buffer(BUFFER_DIST)
    gdf_buffered_ll = gdf_buffered.to_crs('EPSG:4326')

    fig, ax = plt.subplots(figsize=(14, 12))

    # 1. Vẽ vùng Buffer (Màu xám nhạt, nét đứt)
    gdf_buffered_ll.plot(ax=ax, facecolor='lightgray', edgecolor='gray',
                         linestyle='--', linewidth=1.5, alpha=0.4,
                         label=f'Vùng Buffer (+{BUFFER_DIST:.0f}m)')

    # 2. Vẽ các ô H3 được chọn (Màu xanh nhạt)
    h3_grid.plot(ax=ax, facecolor='lightblue', edgecolor='blue',
                 alpha=0.6, linewidth=0.5, label=f'H3 Final ({len(h3_grid)} ô)')

    # 3. Vẽ ranh giới GỐC (Màu đỏ đậm)
    roi_gdf.plot(ax=ax, facecolor='none', edgecolor='red',
                 linewidth=2.5, label='Ranh giới gốc (13 tỉnh ĐBSCL)')

    ax.set_title(f"Quy trình H3 Grid Generation: Buffer → Polyfill → Validate\n"
                 f"(Resolution {RES} ~ {edge_len:.1f}m/cạnh | Buffer {BUFFER_DIST:.0f}m)",
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('Longitude', fontsize=11)
    ax.set_ylabel('Latitude', fontsize=11)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()
    print("✅ Visualization hoàn tất!")
else:
    print("⚠️ Chưa có h3_grid. Vui lòng chạy phần 3.1 trước!")

🎨 Vẽ biểu đồ quá trình H3 grid generation...


NameError: name 'plt' is not defined

In [ ]:
# ============================================================
# PHẦN 3: HÀM XỬ LÝ - Các hàm hỗ trợ xử lý raster
# ============================================================

# LANDCOVER CLASSES (ESA WorldCover - đầy đủ 11 lớp)
LANDCOVER_CLASSES = {
    10: 'tree', 20: 'shrub', 30: 'grass', 40: 'crop',
    50: 'built', 60: 'bare', 70: 'snow_ice',
    80: 'water', 90: 'wetland', 95: 'mangrove', 100: 'moss_lichen'
}

def process_raster_by_polygon(gdf, tiff_path, process_type='landcover'):
    """
    Xử lý raster theo polygon H3 (LandCover | River | DEM | Slope) - Tối ưu

    4 raster xử lý độc lập:
    - LandCover: tính % các lớp
    - River: tính % pixel có sông (feature riêng)
    - DEM: tính min/max/mean elevation
    - Slope: tính min/max/mean độ dốc + tuỳ chọn % pixel có độ dốc > 1 độ
    """
    with rasterio.open(tiff_path) as src:
        gdf_crs = gdf.to_crs(src.crs)

    print(f"⚡ Xử lý {process_type}...")
    results = {}

    with rasterio.open(tiff_path) as src:
        for geom in gdf_crs.geometry:
            try:
                masked, _ = rasterio_mask(src, [geom], crop=True)
                data = masked[0].flatten()

                if process_type == 'landcover':
                    data_valid = data[data > 0]
                    if len(data_valid) > 0:
                        for cls_id, cls_name in LANDCOVER_CLASSES.items():
                            col = f'pct_{cls_name}'
                            if col not in results:
                                results[col] = []
                            pct = (np.sum(data_valid == cls_id) / len(data_valid) * 100)
                            results[col].append(pct)
                    else:
                        for cls_name in LANDCOVER_CLASSES.values():
                            col = f'pct_{cls_name}'
                            if col not in results:
                                results[col] = []
                            results[col].append(0)

                elif process_type == 'river':
                    pct = (np.sum(data > 0) / len(data) * 100) if len(data) > 0 else 0
                    if 'pct_water_river' not in results:
                        results['pct_water_river'] = []
                    results['pct_water_river'].append(pct)

                elif process_type == 'dem':
                    data_valid = data[(data != -32768) & ~np.isnan(data)]
                    if len(data_valid) > 0:
                        for key in ['dem_min', 'dem_max', 'dem_mean']:
                            if key not in results:
                                results[key] = []
                        results['dem_min'].append(float(np.nanmin(data_valid)))
                        results['dem_max'].append(float(np.nanmax(data_valid)))
                        results['dem_mean'].append(float(np.nanmean(data_valid)))
                    else:
                        for key in ['dem_min', 'dem_max', 'dem_mean']:
                            if key not in results:
                                results[key] = []
                            results[key].append(0)

                elif process_type == 'slope':
                    # Slope: tính min/max/mean + tuỳ chọn % pixel có độ dốc > 1 độ
                    data_valid = data[(data != -32768) & ~np.isnan(data) & (data >= 0)]
                    if len(data_valid) > 0:
                        for key in ['slope_min', 'slope_max', 'slope_mean', 'pct_slope_gt_1deg']:
                            if key not in results:
                                results[key] = []

                        results['slope_min'].append(float(np.nanmin(data_valid)))
                        results['slope_max'].append(float(np.nanmax(data_valid)))
                        results['slope_mean'].append(float(np.nanmean(data_valid)))

                        # Tính % pixel có độ dốc > 1 độ
                        pct_gt_1deg = (np.sum(data_valid > 1) / len(data_valid) * 100)
                        results['pct_slope_gt_1deg'].append(pct_gt_1deg)
                    else:
                        for key in ['slope_min', 'slope_max', 'slope_mean', 'pct_slope_gt_1deg']:
                            if key not in results:
                                results[key] = []
                            results[key].append(0)

                elif process_type == 'salinity':
                    # Salinity: tính min/max/mean/std + % pixel theo các ngưỡng cụ thể
                    # Giá trị hợp lệ: [-100, 100] ppt
                    data_valid = data[(data >= -100) & (data <= 100) & ~np.isnan(data)]
                    if len(data_valid) > 0:
                        for key in ['salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std',
                                   'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']:
                            if key not in results:
                                results[key] = []

                        results['salinity_min'].append(float(np.nanmin(data_valid)))
                        results['salinity_max'].append(float(np.nanmax(data_valid)))
                        results['salinity_mean'].append(float(np.nanmean(data_valid)))
                        results['salinity_std'].append(float(np.nanstd(data_valid)))

                        # Tính % pixel có dữ liệu độ mặn (valid pixels)
                        total_pixels = data.size
                        valid_pixels = len(data_valid)
                        pct_valid = (valid_pixels / total_pixels * 100) if total_pixels > 0 else 0
                        results['pct_salinity_pixels'].append(pct_valid)

                        # Tính % pixel theo các ngưỡng cụ thể (để phân cấp rủi ro)
                        pct_gte_0_2 = (np.sum(data_valid >= 0.2) / len(data_valid) * 100)
                        pct_gte_0_5 = (np.sum(data_valid >= 0.5) / len(data_valid) * 100)
                        pct_gte_1_0 = (np.sum(data_valid >= 1.0) / len(data_valid) * 100)

                        results['pct_salinity_gte_0_2'].append(pct_gte_0_2)
                        results['pct_salinity_gte_0_5'].append(pct_gte_0_5)
                        results['pct_salinity_gte_1_0'].append(pct_gte_1_0)
                    else:
                        for key in ['salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std',
                                   'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']:
                            if key not in results:
                                results[key] = []
                            results[key].append(0)
            except:
                if process_type == 'landcover':
                    for cls_name in LANDCOVER_CLASSES.values():
                        col = f'pct_{cls_name}'
                        if col not in results:
                            results[col] = []
                        results[col].append(0)
                elif process_type == 'river':
                    if 'pct_water_river' not in results:
                        results['pct_water_river'] = []
                    results['pct_water_river'].append(0)
                elif process_type == 'dem':
                    for key in ['dem_min', 'dem_max', 'dem_mean']:
                        if key not in results:
                            results[key] = []
                        results[key].append(0)
                elif process_type == 'slope':
                    for key in ['slope_min', 'slope_max', 'slope_mean', 'pct_slope_gt_1deg']:
                        if key not in results:
                            results[key] = []
                        results[key].append(0)
                elif process_type == 'salinity':
                    for key in ['salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std',
                               'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']:
                        if key not in results:
                            results[key] = []
                        results[key].append(0)

    for col, vals in results.items():
        gdf[col] = vals
    return gdf



### 3.3 Chạy xử lý static features

In [ ]:
# BƯỚC 1: GÁN NHÃN LANDCOVER (River riêng, không tính vào tổng)
def assign_landcover_labels(gdf, water_threshold=50, land_threshold=50):
    """
    Gán nhãn landcover dựa trên % các lớp LandCover (B1 của "chiến lược sếp")

    Dữ liệu đầu vào: 9 lớp LandCover + River (feature riêng)
    - Tree, Shrub, Grass, Crop, Built, Bare, Water (từ LandCover)
    - River (pct_water_river) là feature riêng, KHÔNG bao gồm trong tổng
    - is_sea, no_data_land là nhãn (label) để QC/lọc, không ảnh hưởng dữ liệu

    Logic phân lớp:
    - Ô biển (is_sea=True) → 'Sea'
    - Ô không dữ liệu (no_data_land=True) → 'No_Data_Land'
    - Ô nước chính (pct_water >= threshold) → 'Water-dominant'
    - Ô đất liền (tổng đất >= threshold) → 'Land-dominant'
    - Còn lại → 'Mixed'
    """
    def classify_cell(row):
        if row.get('is_sea', False):
            return 'Sea'
        if row.get('no_data_land', False):
            return 'No_Data_Land'

        pct_water = row.get('pct_water', 0)  # Chỉ LandCover water, không bao gồm river
        pct_land = sum(row.get(f'pct_{cls}', 0) for cls in
                      ['crop', 'built', 'tree', 'shrub', 'grass', 'bare'])

        if pct_water >= water_threshold:
            return 'Water-dominant'
        elif pct_land >= land_threshold:
            return 'Land-dominant'
        else:
            return 'Mixed'

    gdf['landcover_label'] = gdf.apply(classify_cell, axis=1)
    print("📊 Phân phối nhãn landcover ban đầu:")
    for label, count in gdf['landcover_label'].value_counts().items():
        pct = (count / len(gdf)) * 100
        print(f"   {label}: {count} ô ({pct:.1f}%)")
    return gdf

# BƯỚC 2: HIỆU CHỈNH NHÃN BẰNG NGỮ CẢNH KHÔNG GIAN
def refine_landcover_labels_by_neighbors(gdf, water_threshold_refine=50):
    """
    Hiệu chỉnh nhãn landcover dựa trên context của ô lân cận

    Nếu ô Land-dominant nhưng đa số ô lân cận (7 gần nhất) là Water-dominant
    → Chỉnh lại thành Mixed

    Nếu ô Water-dominant nhưng đa số ô lân cận (7 gần nhất) là Land-dominant
    → Chỉnh lại thành Mixed (loại bỏ ô nước nhỏ, rìa biển, cửa sông)
    """
    coords_array = np.array([[p.x, p.y] for p in gdf.geometry.centroid])
    tree = cKDTree(coords_array)

    refined_labels = []
    modified_count = 0

    for idx, (_, row) in enumerate(gdf.iterrows()):
        label = row['landcover_label']

        if label == 'Water-dominant':
            _, indices = tree.query(coords_array[idx], k=7)
            neighbor_labels = gdf.iloc[indices[1:]]['landcover_label'].tolist()
            land_neighbor_pct = (sum(1 for l in neighbor_labels if l == 'Land-dominant')
                                / len(neighbor_labels) * 100)

            if land_neighbor_pct >= water_threshold_refine:
                refined_labels.append('Mixed')
                modified_count += 1
            else:
                refined_labels.append(label)
        else:
            refined_labels.append(label)

    gdf['landcover_label_refined'] = refined_labels
    print("\n📊 Phân phối nhãn sau hiệu chỉnh:")
    for label, count in gdf['landcover_label_refined'].value_counts().items():
        pct = (count / len(gdf)) * 100
        print(f"   {label}: {count} ô ({pct:.1f}%)")
    print(f"\n✏️  Số ô được hiệu chỉnh: {modified_count} ({(modified_count/len(gdf)*100):.1f}%)")
    return gdf

In [ ]:
# BƯỚC 3: PHÂN CẤP RỦI RO SALINITY
def classify_salinity_risk(gdf):
    """
    Phân loại rủi ro salinity dựa trên tỷ lệ pixel với các ngưỡng cụ thể:

    Low:    ≥30% pixel ≥ 0.2 ppt
    Medium: ≥50% pixel ≥ 0.5 ppt
    High:   ≥50% pixel ≥ 1.0 ppt
    None:   Không đủ điều kiện
    """
    def classify_risk(row):
        # Nếu không có dữ liệu salinity, trả về 'No Data'
        if pd.isna(row.get('pct_salinity_gte_0_2', np.nan)):
            return 'No Data'

        pct_gte_1_0 = row.get('pct_salinity_gte_1_0', 0)
        pct_gte_0_5 = row.get('pct_salinity_gte_0_5', 0)
        pct_gte_0_2 = row.get('pct_salinity_gte_0_2', 0)

        # Ưu tiên từ cao đến thấp
        if pct_gte_1_0 >= 50:
            return 'High'
        elif pct_gte_0_5 >= 50:
            return 'Medium'
        elif pct_gte_0_2 >= 30:
            return 'Low'
        else:
            return 'None'

    gdf['salinity_risk'] = gdf.apply(classify_risk, axis=1)
    print("📊 Phân cấp rủi ro Salinity:")
    for risk, count in gdf['salinity_risk'].value_counts().items():
        pct = (count / len(gdf)) * 100
        print(f"   {risk}: {count} ô ({pct:.1f}%)")
    return gdf


In [ ]:
# ============================================================
# 3.3 CHẠY XỬ LÝ STATIC FEATURES: LandCover + River + DEM + Slope + Salinity
# Sử dụng H3 grid ĐBSCL đã tạo từ section 3.1 (KHÔNG phụ thuộc rain/temp)
# ============================================================

if os.path.exists(landcover_path) and os.path.exists(river_path) and os.path.exists(dem_path):
    if 'h3_grid' in locals() and not h3_grid.empty:
        # Sử dụng H3 grid ĐBSCL đã tạo sẵn
        h3_gdf = h3_grid.copy()
        print(f"📍 Sử dụng H3 grid {len(h3_gdf)} ô từ ranh giới 13 tỉnh (section 3.1)")

        print("\n=== XỬ LÝ RASTER ĐỘC LẬP ===")
        h3_gdf = process_raster_by_polygon(h3_gdf, landcover_path, 'landcover')
        h3_gdf = process_raster_by_polygon(h3_gdf, river_path, 'river')
        h3_gdf = process_raster_by_polygon(h3_gdf, dem_path, 'dem')

        # Thêm Slope nếu có sẵn
        if os.path.exists(slope_path):
            h3_gdf = process_raster_by_polygon(h3_gdf, slope_path, 'slope')
            print("✅ Hoàn tất xử lý Slope!")
        else:
            print("⏳ Slope chưa có")
            print(f"💡 Thêm Slope sau khi GEE export xong: {slope_path}")
            # Khởi tạo các cột Slope = NaN (để tránh lỗi ở bước sau)
            for col in ['slope_min', 'slope_max', 'slope_mean', 'pct_slope_gt_1deg']:
                h3_gdf[col] = np.nan

        # Thêm Salinity nếu có sẵn
        if os.path.exists(salinity_path):
            h3_gdf = process_raster_by_polygon(h3_gdf, salinity_path, 'salinity')
            print("✅ Hoàn tất xử lý Salinity!")
        else:
            print("⏳ Salinity chưa có")
            print(f"💡 Thêm Salinity từ DoMan_DBSCL_2023.tif: {salinity_path}")
            # Khởi tạo các cột Salinity = NaN
            for col in ['salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std',
                       'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']:
                h3_gdf[col] = np.nan

        rasters_count = 3  # landcover, river, dem
        rasters_name = "LandCover + River + DEM"
        if os.path.exists(slope_path):
            rasters_count += 1
            rasters_name += " + Slope"
        if os.path.exists(salinity_path):
            rasters_count += 1
            rasters_name += " + Salinity"
        print(f"✅ Hoàn tất xử lý {rasters_count} rasters ({rasters_name})!")

        print("\n=== TẠO CỘT PHỤ TRỢ ===")
        # Danh sách explicit các lớp landcover (loại bỏ pct_slope_gt_1deg)
        landcover_cols = [
            'pct_tree', 'pct_shrub', 'pct_grass', 'pct_crop', 'pct_built',
            'pct_bare', 'pct_water', 'pct_wetland', 'pct_mangrove',
            'pct_snow_ice', 'pct_moss_lichen'  # 11 lớp đầy đủ
        ]
        # Chỉ tính các cột có trong h3_gdf (tránh lỗi KeyError nếu chưa có)
        landcover_cols = [col for col in landcover_cols if col in h3_gdf.columns]
        h3_gdf['total_landcover_pct'] = h3_gdf[landcover_cols].sum(axis=1)
        h3_gdf['no_data_land'] = (h3_gdf['total_landcover_pct'] == 0)
        h3_gdf['is_sea'] = (h3_gdf['pct_water'] >= 80) & (h3_gdf['pct_mangrove'] < 5)

        sea_mask = h3_gdf['is_sea'] == True
        h3_gdf.loc[sea_mask, ['dem_min', 'dem_max', 'dem_mean']] = np.nan
        h3_gdf.loc[sea_mask, ['slope_min', 'slope_max', 'slope_mean', 'pct_slope_gt_1deg']] = np.nan
        h3_gdf.loc[sea_mask, ['pct_water_river']] = np.nan

        # Nếu có salinity, cũng mask cho ô biển
        h3_gdf.loc[sea_mask, ['salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std',
                                  'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']] = np.nan

        print("\n=== GÁN NHÃN LANDCOVER (B1) ===")
        h3_gdf = assign_landcover_labels(h3_gdf, water_threshold=50, land_threshold=50)

        print("\n=== HIỆU CHỈNH NHÃN LANDCOVER (B2) ===")
        h3_gdf = refine_landcover_labels_by_neighbors(h3_gdf, water_threshold_refine=50)
        # Phân cấp rủi ro salinity nếu có dữ liệu
        if 'pct_salinity_gte_0_2' in h3_gdf.columns:
            print("\n=== PHÂN CẤP RỦI RO SALINITY ===")
            h3_gdf = classify_salinity_risk(h3_gdf)

        print("\n✅ Hoàn tất xử lý static features!")
    else:
        print("⚠️ H3 grid chưa được tạo. Vui lòng chạy section 3.1 trước!")
else:
    missing_files = []
    if not os.path.exists(landcover_path):
        missing_files.append('LandCover_DBSCL_2021.tif')
    if not os.path.exists(river_path):
        missing_files.append('River_Water_DBSCL_MaxExtent.tif')
    if not os.path.exists(dem_path):
        missing_files.append('DEM_DBSCL.tif')
    if not os.path.exists(slope_path):
        missing_files.append('Slope_DBSCL.tif')
    if not os.path.exists(salinity_path):
        missing_files.append('DoMan_DBSCL_2023.tif (Salinity - tùy chọn)')
    print(f"⚠️ Thiếu file dữ liệu: {', '.join(missing_files)}")

📍 Sử dụng H3 grid 6959 ô từ ranh giới 13 tỉnh (section 3.1)

=== XỬ LÝ RASTER ĐỘC LẬP ===
⚡ Xử lý landcover...
⚡ Xử lý river...
⚡ Xử lý dem...
⚡ Xử lý slope...
✅ Hoàn tất xử lý Slope!
⚡ Xử lý salinity...
✅ Hoàn tất xử lý Salinity!
✅ Hoàn tất xử lý 5 rasters (LandCover + River + DEM + Slope + Salinity)!

=== TẠO CỘT PHỤ TRỢ ===

=== GÁN NHÃN LANDCOVER (B1) ===
📊 Phân phối nhãn landcover ban đầu:
   Land-dominant: 5885 ô (84.6%)
   Mixed: 495 ô (7.1%)
   Water-dominant: 494 ô (7.1%)
   Sea: 84 ô (1.2%)
   No_Data_Land: 1 ô (0.0%)

=== HIỆU CHỈNH NHÃN LANDCOVER (B2) ===


/tmp/ipython-input-1993830690.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords_array = np.array([[p.x, p.y] for p in gdf.geometry.centroid])



📊 Phân phối nhãn sau hiệu chỉnh:
   Land-dominant: 5885 ô (84.6%)
   Mixed: 681 ô (9.8%)
   Water-dominant: 308 ô (4.4%)
   Sea: 84 ô (1.2%)
   No_Data_Land: 1 ô (0.0%)

✏️  Số ô được hiệu chỉnh: 186 (2.7%)

=== PHÂN CẤP RỦI RO SALINITY ===
📊 Phân cấp rủi ro Salinity:
   Medium: 5736 ô (82.4%)
   Low: 1138 ô (16.4%)
   No Data: 84 ô (1.2%)
   None: 1 ô (0.0%)

✅ Hoàn tất xử lý static features!


### 3.4 Kiểm tra chất lượng (QC)

## Soft Features: Thêm Ngữ Cảnh Không Gian (Neighbor Water Context)

Thay vì gán class "gần sông" (binary), chúng ta thêm 2 **soft features** số để giữ thông tin mềm hơn:

- **`pct_water_neighbor_avg`**: Trung bình % nước (sông) từ 6 ô H3 hàng xóm gần nhất
  - Giá trị: [0, 100] (%)
  - Ý nghĩa: Nếu ô này có giá trị cao → ô xung quanh có nhiều sông/nước

- **`pct_water_neighbor_max`**: % nước lớn nhất (sông) từ 6 ô H3 hàng xóm gần nhất
  - Giá trị: [0, 100] (%)
  - Ý nghĩa: Nếu ô này có giá trị cao → ít nhất 1 ô hàng xóm có rất nhiều sông

**Lợi ích:**
- Giữ thông tin numeric (không mất dữ liệu so với binary class)
- Có thể dùng trực tiếp cho ML models (regression, tree-based)
- Tránh gán class cứng nhắc, cho phép model học mối quan hệ phức tạp

In [ ]:
if 'h3_gdf' in locals() and not h3_gdf.empty:
    print("\n=== KIỂM TRA CHẤT LƯỢNG ===\n")

    # QC 1: Landcover coverage trên đất liền (11 lớp landcover, không bao gồm River & Slope)
    valid_cells = h3_gdf[~h3_gdf['is_sea']]

    # Thống kê tổng landcover % (NẾN ~100% ±1-2%)
    lc_mean = valid_cells['total_landcover_pct'].mean()
    lc_median = valid_cells['total_landcover_pct'].median()
    lc_std = valid_cells['total_landcover_pct'].std()
    print(f"📊 KIỂM TRA CHÍNH: Tổng LandCover % (11 lớp):")
    print(f"   Mean: {lc_mean:.2f}% | Median: {lc_median:.2f}% | Std: {lc_std:.2f}%")
    if 98 < lc_mean < 102:
        print(f"   ✅ OK - Gần 100% (sai số ±{abs(100-lc_mean):.2f}%)")
    else:
        print(f"   ⚠️ LỆCH - Sai số {abs(100-lc_mean):.2f}% > 2%")

    # Kiểm tra ô có tổng lệch lớn
    invalid_coverage = valid_cells[(valid_cells['total_landcover_pct'] < 80) |
                                   (valid_cells['total_landcover_pct'] > 120)]
    invalid_coverage = invalid_coverage[~invalid_coverage['no_data_land']]  # Loại bỏ no_data

    print(f"\n⚠️ Ô H3 đất liền có tổng LandCover LỆCH do raster mask: "
          f"{len(invalid_coverage)} ({len(invalid_coverage)/len(valid_cells)*100:.1f}%)")
    if len(invalid_coverage) > 0:
        display(invalid_coverage[['total_landcover_pct', 'no_data_land']].head())

    # QC 2: DEM stats (dùng .dropna() để tránh NaN khi toàn bộ bị mask)
    print(f"\n📊 Thống kê DEM:")
    dem_valid = h3_gdf['dem_mean'].dropna()
    if len(dem_valid) > 0:
        print(f"   DEM_mean: {dem_valid.min():.2f} - {dem_valid.max():.2f} m (n={len(dem_valid)} ô có DEM)")
    else:
        print(f"   DEM_mean: TOÀN BỘ NaN (tất cả ô bị mask vì biển)")

    # QC 2b: Slope stats (mới thêm)
    print(f"\n📊 Thống kê Slope:")
    slope_valid = h3_gdf['slope_mean'].dropna()
    if len(slope_valid) > 0:
        print(f"   Slope_mean: {slope_valid.min():.2f} - {slope_valid.max():.2f} độ (n={len(slope_valid)} ô có Slope)")
        pct_slope_gt_1 = h3_gdf['pct_slope_gt_1deg'].dropna()
        if len(pct_slope_gt_1) > 0:
            print(f"   % Slope > 1°: {pct_slope_gt_1.mean():.1f}% (±{pct_slope_gt_1.std():.1f}%)")
    else:
        print(f"   Slope_mean: TOÀN BỘ NaN (tất cả ô bị mask vì biển)")

    sea_count = h3_gdf['is_sea'].sum()
    land_count = len(valid_cells)
    print(f"\n🌊 Ô H3 biển (pct_water >= 80 & pct_mangrove < 5): {sea_count} ({sea_count/len(h3_gdf)*100:.1f}%)")
    print(f"🏞️  Ô H3 đất liền: {land_count} ({land_count/len(h3_gdf)*100:.1f}%)")

    # QC 3: LandCover stats (11 lớp, không River & không Slope)
    lc_cols = [
        'pct_tree', 'pct_shrub', 'pct_grass', 'pct_crop', 'pct_built',
        'pct_bare', 'pct_water', 'pct_wetland', 'pct_mangrove',
        'pct_snow_ice', 'pct_moss_lichen'  # 11 lớp landcover
    ]
    # Chỉ lấy các cột có trong h3_gdf
    lc_cols = [col for col in lc_cols if col in h3_gdf.columns]
    print(f"\n📊 Land Cover stats (đất liền):")
    for col in sorted(lc_cols):
        print(f"   {col}: {valid_cells[col].mean():.1f}% (±{valid_cells[col].std():.1f}%)")

    # QC 4: River (feature riêng)
    if 'pct_water_river' in h3_gdf.columns:
        river_pct = h3_gdf['pct_water_river'].mean()
        print(f"\n🌊 River (feature riêng): {river_pct:.1f}%")

    # QC 5: no_data_land
    no_data_count = h3_gdf['no_data_land'].sum()
    print(f"\n⚠️  Ô H3 no_data_land: {no_data_count} ({no_data_count/len(h3_gdf)*100:.1f}%) - Không bất thường")

    # QC 5b: SALINITY stats (mới thêm - độ mặn)
    print(f"\n📊 Thống kê SALINITY (ĐỘ MẶN):")
    if 'salinity_mean' in h3_gdf.columns:
        salinity_valid = h3_gdf['salinity_mean'].dropna()
        if len(salinity_valid) > 0:
            print(f"   Salinity_mean: {salinity_valid.min():.2f} - {salinity_valid.max():.2f} ppt")
            print(f"   Salinity_mean (avg): {salinity_valid.mean():.2f} ± {salinity_valid.std():.2f} ppt (n={len(salinity_valid)} ô)")

            # Thống kê % pixel có dữ liệu độ mặn
            if 'pct_salinity_pixels' in h3_gdf.columns:
                pct_valid_pixels = h3_gdf['pct_salinity_pixels'].dropna()
                if len(pct_valid_pixels) > 0:
                    print(f"   % Pixel có dữ liệu: {pct_valid_pixels.mean():.1f}% ± {pct_valid_pixels.std():.1f}%")

            # Thống kê % pixel theo các ngưỡng cụ thể
            if 'pct_salinity_gte_0_2' in h3_gdf.columns:
                pct_gte_0_2 = h3_gdf['pct_salinity_gte_0_2'].dropna()
                pct_gte_0_5 = h3_gdf['pct_salinity_gte_0_5'].dropna()
                pct_gte_1_0 = h3_gdf['pct_salinity_gte_1_0'].dropna()

                if len(pct_gte_0_2) > 0:
                    print(f"   % Pixel ≥ 0.2 ppt: {pct_gte_0_2.mean():.1f}% ± {pct_gte_0_2.std():.1f}%")
                if len(pct_gte_0_5) > 0:
                    print(f"   % Pixel ≥ 0.5 ppt: {pct_gte_0_5.mean():.1f}% ± {pct_gte_0_5.std():.1f}%")
                if len(pct_gte_1_0) > 0:
                    print(f"   % Pixel ≥ 1.0 ppt: {pct_gte_1_0.mean():.1f}% ± {pct_gte_1_0.std():.1f}%")

                # Phân cấp rủi ro
                if 'salinity_risk' in h3_gdf.columns:
                    print(f"\n   Phân cấp rủi ro Salinity:")
                    for risk, count in h3_gdf['salinity_risk'].value_counts().items():
                        pct = (count / len(h3_gdf)) * 100
                        print(f"      {risk}: {count} ô ({pct:.1f}%)")
        else:
            print(f"   ⚠️ TOÀN BỘ NaN - Không có dữ liệu độ mặn hợp lệ")
    else:
        print(f"   ⏳ Chưa xử lý dữ liệu độ mặn (chạy section 3.3b trước)")

    # QC 6: LandCover labels
    print(f"\n📊 Phân phối nhãn landcover:")
    if 'landcover_label_refined' in h3_gdf.columns:
        for label, count in h3_gdf['landcover_label_refined'].value_counts().items():
            pct = (count / len(h3_gdf)) * 100
            print(f"   {label}: {count} ô ({pct:.1f}%)")
    elif 'landcover_label' in h3_gdf.columns:
        for label, count in h3_gdf['landcover_label'].value_counts().items():

            pct = (count / len(h3_gdf)) * 100

            print(f"   {label}: {count} ô ({pct:.1f}%)")

    print(f"\n✅ QC hoàn tất! (bao gồm Slope + Salinity)")


=== KIỂM TRA CHẤT LƯỢNG ===

📊 KIỂM TRA CHÍNH: Tổng LandCover % (11 lớp):
   Mean: 99.99% | Median: 100.00% | Std: 1.21%
   ✅ OK - Gần 100% (sai số ±0.01%)

⚠️ Ô H3 đất liền có tổng LandCover LỆCH do raster mask: 0 (0.0%)

📊 Thống kê DEM:
   DEM_mean: -1.94 - 285.80 m (n=6875 ô có DEM)

📊 Thống kê Slope:
   Slope_mean: 0.00 - 14.69 độ (n=6875 ô có Slope)
   % Slope > 1°: 53.7% (±10.1%)

🌊 Ô H3 biển (pct_water >= 80 & pct_mangrove < 5): 84 (1.2%)
🏞️  Ô H3 đất liền: 6875 (98.8%)

📊 Land Cover stats (đất liền):
   pct_bare: 1.2% (±2.6%)
   pct_built: 4.5% (±6.9%)
   pct_crop: 48.9% (±33.6%)
   pct_grass: 3.5% (±5.7%)
   pct_mangrove: 3.3% (±14.4%)
   pct_moss_lichen: 0.0% (±0.0%)
   pct_shrub: 0.3% (±0.9%)
   pct_snow_ice: 0.0% (±0.0%)
   pct_tree: 25.0% (±24.0%)
   pct_water: 12.1% (±19.0%)
   pct_wetland: 1.2% (±3.5%)

🌊 River (feature riêng): 38.7%

⚠️  Ô H3 no_data_land: 1 (0.0%) - Không bất thường

📊 Thống kê SALINITY (ĐỘ MẶN):
   Salinity_mean: 0.13 - 0.89 ppt
   Salinity_mean (avg

In [ ]:
if 'h3_gdf' in locals() and not h3_gdf.empty:
    print("\n=== THÊM SOFT FEATURES: HÀNG XÓM NƯỚC ===\n")

    # ===== TÍNH TOÁN SOFT FEATURES: NEIGHBOR WATER CONTEXT =====
    # Tư tưởng: Chuyển "gần sông" từ class thành các feature mềm (soft features)
    # Thay vì gán nhãn "near_river" (class), thêm 2 feature số:
    # 1. pct_water_neighbor_avg: Trung bình % nước (river) của 7 hàng xóm gần nhất
    # 2. pct_water_neighbor_max: % nước lớn nhất trong 7 hàng xóm gần nhất

    coords_array = np.array([[p.x, p.y] for p in h3_gdf.geometry.centroid])
    tree = cKDTree(coords_array)

    # Khởi tạo các cột mới
    h3_gdf['pct_water_neighbor_avg'] = np.nan
    h3_gdf['pct_water_neighbor_max'] = np.nan

    # Tính cho mỗi ô H3
    for idx, (_, row) in enumerate(h3_gdf.iterrows()):
        # Tìm 7 ô hàng xóm gần nhất (k=7 bao gồm chính nó + 6 hàng xóm)
        _, neighbor_indices = tree.query(coords_array[idx], k=7)

        # Lấy % nước (river) của các hàng xóm (bỏ chính nó, chỉ lấy 6 hàng xóm)
        neighbor_water_values = h3_gdf.iloc[neighbor_indices[1:]]['pct_water_river'].values

        # Loại bỏ NaN (nếu có)
        neighbor_water_valid = neighbor_water_values[~np.isnan(neighbor_water_values)]

        if len(neighbor_water_valid) > 0:
            h3_gdf.iloc[idx, h3_gdf.columns.get_loc('pct_water_neighbor_avg')] = \
                float(np.mean(neighbor_water_valid))
            h3_gdf.iloc[idx, h3_gdf.columns.get_loc('pct_water_neighbor_max')] = \
                float(np.max(neighbor_water_valid))
        else:
            # Nếu không có hàng xóm nào có dữ liệu, đặt thành 0 hoặc NaN
            h3_gdf.iloc[idx, h3_gdf.columns.get_loc('pct_water_neighbor_avg')] = 0
            h3_gdf.iloc[idx, h3_gdf.columns.get_loc('pct_water_neighbor_max')] = 0

    print("✅ Hoàn tất tính soft features hàng xóm:")
    print(f"   - pct_water_neighbor_avg: Trung bình % nước từ 6 hàng xóm gần nhất")
    print(f"   - pct_water_neighbor_max: % nước lớn nhất từ 6 hàng xóm gần nhất")
    print(f"\n📊 Thống kê soft features:")
    print(f"   pct_water_neighbor_avg: Min={h3_gdf['pct_water_neighbor_avg'].min():.2f}%, "
          f"Mean={h3_gdf['pct_water_neighbor_avg'].mean():.2f}%, "
          f"Max={h3_gdf['pct_water_neighbor_avg'].max():.2f}%")
    print(f"   pct_water_neighbor_max: Min={h3_gdf['pct_water_neighbor_max'].min():.2f}%, "
          f"Mean={h3_gdf['pct_water_neighbor_max'].mean():.2f}%, "
          f"Max={h3_gdf['pct_water_neighbor_max'].max():.2f}%")
else:
    print("⚠️ Không có h3_gdf. Vui lòng chạy phần 3.3 trước!")



=== THÊM SOFT FEATURES: HÀNG XÓM NƯỚC ===



/tmp/ipython-input-800048719.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords_array = np.array([[p.x, p.y] for p in h3_gdf.geometry.centroid])


✅ Hoàn tất tính soft features hàng xóm:
   - pct_water_neighbor_avg: Trung bình % nước từ 6 hàng xóm gần nhất
   - pct_water_neighbor_max: % nước lớn nhất từ 6 hàng xóm gần nhất

📊 Thống kê soft features:
   pct_water_neighbor_avg: Min=0.04%, Mean=39.08%, Max=68.55%
   pct_water_neighbor_max: Min=0.07%, Mean=50.25%, Max=69.12%


### 3.5 Lưu kết quả cuối cùng

In [ ]:
# ============================================================
# VISUALIZATION: Mapping Raster Data to H3 Grid (Như xuly.ipynb)
# ============================================================
if 'h3_gdf' in locals() and not h3_gdf.empty:
    print("🎨 Vẽ bản đồ dữ liệu raster mapping vào H3 grid (giống xuly.ipynb)...")

    fig, axes = plt.subplots(1, 3, figsize=(20, 6))

    # ========== BẢN ĐỒ 1: % Crop Coverage ==========
    ax1 = axes[0]
    h3_gdf.plot(column='pct_crop', ax=ax1, cmap='YlGn', legend=True,
               edgecolor='black', linewidth=0.2, alpha=0.8,
               legend_kwds={'label': "Crop Coverage (%)", 'shrink': 0.8})
    ax1.set_xlabel('Longitude', fontsize=11)
    ax1.set_ylabel('Latitude', fontsize=11)
    ax1.set_title('Phân phối % Crop (Lúa, Lúa mưa)\nBuffer → Polyfill → Intersects',
                  fontsize=12, fontweight='bold')
    ax1.grid(alpha=0.2)

    # ========== BẢN ĐỒ 2: % Water (từ LandCover) ==========
    ax2 = axes[1]
    h3_gdf.plot(column='pct_water', ax=ax2, cmap='Blues', legend=True,
               edgecolor='black', linewidth=0.2, alpha=0.8,
               legend_kwds={'label': "Water Coverage (%)", 'shrink': 0.8})
    ax2.set_xlabel('Longitude', fontsize=11)
    ax2.set_ylabel('Latitude', fontsize=11)
    ax2.set_title('Phân phối % Water (Nước, Hồ, Mương)\nBuffer → Polyfill → Intersects',
                  fontsize=12, fontweight='bold')
    ax2.grid(alpha=0.2)

    # ========== BẢN ĐỒ 3: % Tree Coverage ==========
    ax3 = axes[2]
    h3_gdf.plot(column='pct_tree', ax=ax3, cmap='Greens', legend=True,
               edgecolor='black', linewidth=0.2, alpha=0.8,
               legend_kwds={'label': "Tree Coverage (%)", 'shrink': 0.8})
    ax3.set_xlabel('Longitude', fontsize=11)
    ax3.set_ylabel('Latitude', fontsize=11)
    ax3.set_title('Phân phối % Tree (Rừng, Dừa, Xoài)\nBuffer → Polyfill → Intersects',
                  fontsize=12, fontweight='bold')
    ax3.grid(alpha=0.2)

    plt.suptitle(f'Mapping Landcover Data to H3 Grid (H3 Res {RES} - {len(h3_gdf)} cells)\n'
                 f'Kỹ thuật: Buffer {BUFFER_DIST:.0f}m → Polyfill → Validate (Intersects)',
                 fontsize=14, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.show()

    print("✅ Raster mapping visualization hoàn tất!")
    print(f"\n📊 So sánh với xuly.ipynb:")
    print(f"   ✅ Cùng phương pháp: Buffer → Polyfill → Intersects")
    print(f"   ✅ Cùng cách vẽ: H3 cells colored by data values")
    print(f"   ✅ Khác: DBSCL xử lý 13 tỉnh MultiPolygon, xuly chỉ 1 polygon")
    print(f"   ✅ Kết quả: {len(h3_gdf)} ô H3 (vs xuly: chỉ vài trăm ô)")
else:
    print("⚠️ Không có dữ liệu h3_gdf. Vui lòng chạy phần 3.3 trước!")

### Kiểm tra trực quan sự giao nhau giữa ô H3 và dữ liệu Raster (Land Cover)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# ============================================================
# VISUALIZATION: VẼ BẢN ĐỒ KIỂM TRA DỮ LIỆU THIẾU (MISSING DATA)
# ============================================================
if 'h3_gdf' in locals() and not h3_gdf.empty:
    print("Dang ve ban do kiem tra du lieu thieu...\n")

    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    from matplotlib.colors import ListedColormap

    # ===== BƯỚC 1: Tạo các cột chỉ báo dữ liệu thiếu =====
    h3_gdf['has_landcover'] = h3_gdf['total_landcover_pct'].notna() & (h3_gdf['total_landcover_pct'] > 0)
    h3_gdf['has_dem'] = h3_gdf['dem_mean'].notna()
    h3_gdf['has_slope'] = h3_gdf['slope_mean'].notna()
    h3_gdf['has_river'] = h3_gdf['pct_water_river'].notna()
    h3_gdf['data_completeness'] = (
        h3_gdf['has_landcover'].astype(int) +
        h3_gdf['has_dem'].astype(int) +
        h3_gdf['has_slope'].astype(int) +
        h3_gdf['has_river'].astype(int)
    )

    # ===== BƯỚC 2: Tạo figure với nhiều bản đồ =====
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.25)

    # ========== BẢN ĐỒ 1: TỔNG QUAN HOÀN THÀNH DỮ LIỆU ==========
    ax1 = fig.add_subplot(gs[0, 0])
    h3_gdf.plot(column='data_completeness', ax=ax1, cmap='viridis',
               legend=True, edgecolor='gray', linewidth=0.3, alpha=0.7,
               legend_kwds={'label': "Data Types Available", 'shrink': 0.8},
               vmin=0, vmax=4)
    roi_gdf.boundary.plot(ax=ax1, color='red', linewidth=2, linestyle='--')
    ax1.set_title('Data Completeness (0-4 types)', fontsize=10)
    ax1.set_xlabel('Longitude', fontsize=9)
    ax1.set_ylabel('Latitude', fontsize=9)
    ax1.grid(alpha=0.2)

    # ========== BẢN ĐỒ 2: LANDCOVER COVERAGE ==========
    ax2 = fig.add_subplot(gs[0, 1])
    h3_gdf.plot(column='has_landcover', ax=ax2, cmap='RdYlGn',
               edgecolor='gray', linewidth=0.3, alpha=0.7, legend=False)
    roi_gdf.boundary.plot(ax=ax2, color='red', linewidth=2, linestyle='--')
    ax2.set_title('Land Cover Data Availability', fontsize=10)
    ax2.set_xlabel('Longitude', fontsize=9)
    ax2.set_ylabel('Latitude', fontsize=9)
    ax2.grid(alpha=0.2)

    # ========== BẢN ĐỒ 3: DEM COVERAGE ==========
    ax3 = fig.add_subplot(gs[0, 2])
    h3_gdf.plot(column='has_dem', ax=ax3, cmap='RdYlGn',
               edgecolor='gray', linewidth=0.3, alpha=0.7, legend=False)
    roi_gdf.boundary.plot(ax=ax3, color='red', linewidth=2, linestyle='--')
    ax3.set_title('DEM (Elevation) Data Availability', fontsize=10)
    ax3.set_xlabel('Longitude', fontsize=9)
    ax3.set_ylabel('Latitude', fontsize=9)
    ax3.grid(alpha=0.2)

    # ========== BẢN ĐỒ 4: SLOPE COVERAGE ==========
    ax4 = fig.add_subplot(gs[1, 0])
    h3_gdf.plot(column='has_slope', ax=ax4, cmap='RdYlGn',
               edgecolor='gray', linewidth=0.3, alpha=0.7, legend=False)
    roi_gdf.boundary.plot(ax=ax4, color='red', linewidth=2, linestyle='--')
    ax4.set_title('Slope Data Availability', fontsize=10)
    ax4.set_xlabel('Longitude', fontsize=9)
    ax4.set_ylabel('Latitude', fontsize=9)
    ax4.grid(alpha=0.2)

    # ========== BẢN ĐỒ 5: RIVER COVERAGE ==========
    ax5 = fig.add_subplot(gs[1, 1])
    h3_gdf.plot(column='has_river', ax=ax5, cmap='RdYlGn',
               edgecolor='gray', linewidth=0.3, alpha=0.7, legend=False)
    roi_gdf.boundary.plot(ax=ax5, color='red', linewidth=2, linestyle='--')
    ax5.set_title('River Data Availability', fontsize=10)
    ax5.set_xlabel('Longitude', fontsize=9)
    ax5.set_ylabel('Latitude', fontsize=9)
    ax5.grid(alpha=0.2)

    # ========== BẢN ĐỒ 6: LANDCOVER % COVERAGE ==========
    ax6 = fig.add_subplot(gs[1, 2])
    h3_gdf.plot(column='total_landcover_pct', ax=ax6, cmap='viridis',
               edgecolor='gray', linewidth=0.3, alpha=0.7,
               legend=True, legend_kwds={'label': "% Coverage", 'shrink': 0.8})
    roi_gdf.boundary.plot(ax=ax6, color='red', linewidth=2, linestyle='--')
    ax6.set_title('Land Cover Percentage (should ~100%)', fontsize=10)
    ax6.set_xlabel('Longitude', fontsize=9)
    ax6.set_ylabel('Latitude', fontsize=9)
    ax6.grid(alpha=0.2)

    # ========== BẢN ĐỒ 7: SEA vs LAND ==========
    ax7 = fig.add_subplot(gs[2, 0])
    h3_gdf['cell_type'] = 'Land'
    h3_gdf.loc[h3_gdf['is_sea'], 'cell_type'] = 'Sea'
    h3_gdf.loc[h3_gdf['no_data_land'], 'cell_type'] = 'No Data'

    color_map = {'Land': 'green', 'Sea': 'blue', 'No Data': 'gray'}
    for cell_type, color in color_map.items():
        mask = h3_gdf['cell_type'] == cell_type
        h3_gdf[mask].plot(ax=ax7, color=color, edgecolor='gray', linewidth=0.2, alpha=0.7, label=cell_type)

    roi_gdf.boundary.plot(ax=ax7, color='red', linewidth=2, linestyle='--')
    ax7.set_title('Sea vs Land Classification', fontsize=10)
    ax7.set_xlabel('Longitude', fontsize=9)
    ax7.set_ylabel('Latitude', fontsize=9)
    ax7.legend(loc='best', fontsize=8)
    ax7.grid(alpha=0.2)

    # ========== BẢN ĐỒ 8: DEM VALUES ==========
    ax8 = fig.add_subplot(gs[2, 1])
    h3_gdf.plot(column='dem_mean', ax=ax8, cmap='terrain',
               edgecolor='gray', linewidth=0.3, alpha=0.7,
               legend=True, legend_kwds={'label': "Elevation (m)", 'shrink': 0.8})
    roi_gdf.boundary.plot(ax=ax8, color='red', linewidth=2, linestyle='--')
    ax8.set_title('Average Elevation (DEM)', fontsize=10)
    ax8.set_xlabel('Longitude', fontsize=9)
    ax8.set_ylabel('Latitude', fontsize=9)
    ax8.grid(alpha=0.2)

    # ========== BẢN ĐỒ 9: SLOPE VALUES ==========
    ax9 = fig.add_subplot(gs[2, 2])
    h3_gdf.plot(column='slope_mean', ax=ax9, cmap='coolwarm',
               edgecolor='gray', linewidth=0.3, alpha=0.7,
               legend=True, legend_kwds={'label': "Slope (degrees)", 'shrink': 0.8})
    roi_gdf.boundary.plot(ax=ax9, color='red', linewidth=2, linestyle='--')
    ax9.set_title('Average Slope (Gradient)', fontsize=10)
    ax9.set_xlabel('Longitude', fontsize=9)
    ax9.set_ylabel('Latitude', fontsize=9)
    ax9.grid(alpha=0.2)

    plt.suptitle(f'Data Quality Check - DBSCL (13 Provinces)\nH3 Grid Resolution {RES} | Total {len(h3_gdf)} cells',
                 fontsize=12, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

    print("Done with maps!\n")

    # ===== BƯỚC 3: IN THỐNG KÊ CHI TIẾT =====
    print("="*70)
    print("DATA QUALITY STATISTICS")
    print("="*70)

    # Thống kê chung
    total_cells = len(h3_gdf)
    land_cells = len(h3_gdf[~h3_gdf['is_sea']])
    sea_cells = len(h3_gdf[h3_gdf['is_sea']])
    no_data_cells = len(h3_gdf[h3_gdf['no_data_land']])

    print(f"\nTotal H3 Cells:")
    print(f"   Total: {total_cells} cells")
    print(f"   Land: {land_cells} cells ({100*land_cells/total_cells:.1f}%)")
    print(f"   Sea: {sea_cells} cells ({100*sea_cells/total_cells:.1f}%)")
    print(f"   No Data (land): {no_data_cells} cells ({100*no_data_cells/total_cells:.1f}%)")

    # Thống kê loại dữ liệu
    print(f"\nData Coverage (Land only):")
    land_only = h3_gdf[~h3_gdf['is_sea']]

    lc_available = land_only['has_landcover'].sum()
    dem_available = land_only['has_dem'].sum()
    slope_available = land_only['has_slope'].sum()
    river_available = land_only['has_river'].sum()

    print(f"   Land Cover: {lc_available}/{len(land_only)} cells ({100*lc_available/len(land_only):.1f}%)")
    print(f"   DEM: {dem_available}/{len(land_only)} cells ({100*dem_available/len(land_only):.1f}%)")
    print(f"   Slope: {slope_available}/{len(land_only)} cells ({100*slope_available/len(land_only):.1f}%)")
    print(f"   River: {river_available}/{len(land_only)} cells ({100*river_available/len(land_only):.1f}%)")

    # Thống kê hoàn thành
    print(f"\nData Completeness (Land only):")
    for i in range(5):
        count = len(land_only[land_only['data_completeness'] == i])
        pct = 100*count/len(land_only)
        label = {0: '(No data)', 1: '(Poor)', 2: '(Fair)', 3: '(Good)', 4: '(Excellent)'}
        print(f"   {i}/4 data types: {count} cells ({pct:.1f}%) {label.get(i, '')}")

    # Thống kê dữ liệu hết
    print(f"\nMissing Data (Land only):")
    missing_lc = len(land_only[~land_only['has_landcover']])
    missing_dem = len(land_only[~land_only['has_dem']])
    missing_slope = len(land_only[~land_only['has_slope']])
    missing_river = len(land_only[~land_only['has_river']])

    if missing_lc > 0:
        print(f"   No Land Cover: {missing_lc} cells ({100*missing_lc/len(land_only):.1f}%)")
    if missing_dem > 0:
        print(f"   No DEM: {missing_dem} cells ({100*missing_dem/len(land_only):.1f}%)")
    if missing_slope > 0:
        print(f"   No Slope: {missing_slope} cells ({100*missing_slope/len(land_only):.1f}%)")
    if missing_river > 0:
        print(f"   No River: {missing_river} cells ({100*missing_river/len(land_only):.1f}%)")

    if missing_lc == 0 and missing_dem == 0 and missing_slope == 0 and missing_river == 0:
        print(f"   ALL LAND CELLS HAVE COMPLETE DATA!")

    print("\n" + "="*70)
    print("Data quality analysis completed!")
    print("="*70 + "\n")

else:
    print("No h3_gdf data found. Please run section 3.3 first!")



Hình ảnh trên hiển thị dữ liệu Land Cover với đường viền màu đỏ của một số ô H3 được vẽ đè lên. Bạn có thể phóng to hình ảnh để kiểm tra kỹ hơn xem các ô H3 có khớp với ranh giới của các vùng dữ liệu raster hay không. Hàm `rasterio.mask.mask` đã được sử dụng để cắt dữ liệu raster theo từng polygon H3, nên về mặt kỹ thuật, dữ liệu được trích xuất đã nằm bên trong các ranh giới H3.

In [ ]:
if 'h3_gdf' in locals() and not h3_gdf.empty:
    # Bỏ cột tạm
    h3_output = h3_gdf.drop(columns=['total_landcover_pct'], errors='ignore')

    # Lưu GeoJSON (giữ geometry)
    geojson_path = os.path.join(data_folder, 'H3_Static_Features.geojson')
    h3_output.to_file(geojson_path, driver='GeoJSON')
    print(f"💾 GeoJSON: {geojson_path}")

    # Lưu CSV (bỏ geometry)
    csv_output_path = os.path.join(data_folder, 'H3_Static_Features.csv')
    h3_output.drop(columns='geometry').to_csv(csv_output_path, index=False)
    print(f"💾 CSV: {csv_output_path}")

    # Hiển thị 5 dòng đầu (với các cột quan trọng, bao gồm soft features + salinity risk)
    print("\n=== KẾT QUẢ (5 dòng đầu) ===")
    display_cols = ['h3_index', 'pct_water', 'pct_crop', 'pct_built', 'pct_tree',
                   'pct_water_river', 'pct_water_neighbor_avg', 'pct_water_neighbor_max',
                   'slope_mean', 'slope_max', 'pct_slope_gt_1deg',
                   'salinity_mean', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0', 'salinity_risk']
    if 'landcover_label' in h3_output.columns:
        display_cols.append('landcover_label')
    if 'landcover_label_refined' in h3_output.columns:
        display_cols.append('landcover_label_refined')

    # Chỉ lấy các cột có tồn tại
    display_cols = [col for col in display_cols if col in h3_output.columns]
    print(h3_output[display_cols].head().to_string())

    print(f"\n✅ HOÀN TẤT! Tổng {len(h3_output)} ô H3 có static features:")
    print(f"   - LandCover (11 lớp) + River (feature riêng)")
    print(f"   - DEM (min/max/mean) + Slope (min/max/mean + % > 1°)")
    print(f"   - Soft Features: Neighbor water context (avg/max)")
    print(f"   - SALINITY: mean/std + % pixel ≥ 0.2/0.5/1.0 ppt + Risk classification")

💾 GeoJSON: /content/drive/MyDrive/Du_Lieu_DBSCL_v2/H3_Static_Features.geojson
💾 CSV: /content/drive/MyDrive/Du_Lieu_DBSCL_v2/H3_Static_Features.csv

=== KẾT QUẢ (5 dòng đầu) ===
          h3_index  pct_water   pct_crop  pct_built   pct_tree  pct_water_river  slope_mean  slope_max  pct_slope_gt_1deg  salinity_mean  pct_salinity_gte_0_2  pct_salinity_gte_0_5  pct_salinity_gte_1_0 salinity_risk landcover_label landcover_label_refined
0  8765a26f5ffffff   0.280443  90.819188   1.667897   5.003690        67.796780    1.810632  12.729647          57.747475       0.576248             99.291101             79.743590                   0.0        Medium   Land-dominant           Land-dominant
1  8765a0353ffffff  29.254613  19.734317   4.073801  27.985240        49.262626    1.794070  16.827198          56.835684       0.546139             97.751961             75.754375                   0.0        Medium   Land-dominant           Land-dominant
2  8765a3c9dffffff   0.559647  88.910162   5.419735

In [ ]:
# Import thêm thư viện cần thiết
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
import matplotlib.pyplot as plt

print("✅ Thư viện đã sẵn sàng cho phân tích khoảng cách đến bờ biển")

In [ ]:
# ============================================================
# TÍNH KHOẢNG CÁCH TỪ Ô H3 ĐẾN RANH GIỚI
# ============================================================

# Tìm biến H3 (có thể là h3_grid hoặc h3_gdf)
h3_data = None
if 'h3_gdf' in locals():
    h3_data = h3_gdf
elif 'h3_grid' in locals():
    h3_data = h3_grid

# Tìm biến ranh giới (có thể là roi_gdf hoặc roi_gdf_clean)
roi_data = None
if 'roi_gdf' in locals():
    roi_data = roi_gdf
elif 'roi_gdf_clean' in locals():
    roi_data = roi_gdf_clean

if h3_data is not None and roi_data is not None:
    # Chuyển CRS sang UTM
    h3_utm = h3_data.to_crs('EPSG:32648')
    roi_utm = roi_data.to_crs('EPSG:32648')
    
    # Boundary: ranh giới gốc (không buffer)
    boundary = roi_utm.geometry.unary_union
    
    # Tính distance tới ranh giới gốc
    h3_data['distance_m'] = h3_utm.geometry.apply(
        lambda geom: geom.centroid.distance(boundary)
    )
    h3_data['distance_km'] = h3_data['distance_m'] / 1000
    
    print("✅ Đã tính distance_m (tới ranh giới gốc)")
    print(f"\n📊 Thống kê:")
    print(f"   Min:  {h3_data['distance_km'].min():.3f} km")
    print(f"   Max:  {h3_data['distance_km'].max():.3f} km")
    print(f"   Mean: {h3_data['distance_km'].mean():.3f} km")
else:
    print("❌ Không tìm thấy dữ liệu!")
    if h3_data is None:
        print("   - Cần h3_gdf hoặc h3_grid từ cell 24")
    if roi_data is None:
        print("   - Cần roi_gdf hoặc roi_gdf_clean từ cell 23")

⚠️  Cần h3_gdf và roi_gdf


## 📋 Pipeline: Tính `distance_m` - Khoảng Cách Từ Ô H3 Đến Ranh Giới

### **Mục Đích:**
Tính **khoảng cách từ tâm mỗi ô H3 đến ranh giới ĐBSCL** (bờ biển/sông biên giới)  
Kết quả = **1 số duy nhất** (đơn vị: mét)

---

### **Dữ Liệu Đầu Vào (Input):**

| Dữ Liệu | Định Nghĩa | Nguồn |
|--------|-----------|-------|
| **h3_gdf** | GeoDataFrame chứa hàng ngàn ô H3 lục giác | Phần 3.1 |
| **roi_gdf** | GeoDataFrame chứa ranh giới 13 tỉnh ĐBSCL | Phần 3.0 |

**Điều kiện:** Cả 2 đều là GeoDataFrame với hệ tọa độ WGS84 (EPSG:4326)

---

## **PIPELINE - 4 Bước Chính:**

### **📍 BƯỚC 1: Chuyển Hệ Tọa Độ Sang Metric**

```
Input:   h3_gdf (WGS84)     roi_gdf (WGS84)
           ↓ to_crs()          ↓ to_crs()
Output:  h3_utm (UTM 48N)   roi_utm (UTM 48N)
```

**Lý do chuyển đổi:**
- WGS84 = tọa độ địa lý (độ, phút, giây)
- UTM = tọa độ metric (mét) → dùng để tính khoảng cách chính xác
- ĐBSCL nằm trong **UTM Zone 48N** (EPSG:32648)

**Kết quả:** Mỗi điểm đều có tọa độ (X_meter, Y_meter)

---

### **🔀 BƯỚC 2: Gộp Ranh Giới Thành 1 Geometry**

```
Input:   roi_utm (có 13 polygon = 13 tỉnh)
           ↓ unary_union()
Output:  boundary (1 polygon/multipolygon duy nhất)
```

**Tại sao gộp?**
- Mỗi tỉnh là polygon riêng
- Gộp lại = 1 ranh giới liên tục cho dễ tính toán
- Chỉ cần 1 lần `distance()` thay vì 13 lần

**Kết quả:** 1 geometry "biên giới ĐBSCL" hoàn chỉnh

---

### **📌 BƯỚC 3: Lấy Centroid (Tâm) Của Mỗi Ô H3**

```
Với mỗi ô H3 lục giác:
    centroid = ô.geometry.centroid
    (tọa độ X, Y của tâm lục giác)
```

**Ý nghĩa:**
- Mỗi ô H3 là đa giác (polygon)
- Centroid = điểm trung tâm của đa giác
- Đại diện vị trí ô H3 bằng **1 điểm duy nhất**

**Kết quả:** Tọa độ (X_meter, Y_meter) của tâm mỗi ô H3

---

### **🎯 BƯỚC 4: Tính Khoảng Cách Từ Centroid Đến Ranh Giới**

```
Với mỗi ô H3:
    distance_m = centroid.distance(boundary)
```

**Công thức (Euclidean Distance):**
$$d = \sqrt{(X_{centroid} - X_{boundary})^2 + (Y_{centroid} - Y_{boundary})^2}$$

**Giải thích:**
- `.distance()` = hàm Shapely tính khoảng cách **ngắn nhất** từ 1 điểm đến 1 geometry
- Nếu điểm nằm **trên** ranh giới → khoảng cách = 0
- Nếu điểm nằm **trong** ranh giới → khoảng cách > 0 (đến ranh giới gần nhất)
- Nếu điểm nằm **ngoài** ranh giới → khoảng cách > 0 (đến ranh giới gần nhất)

**Kết quả:** 1 số dương (đơn vị: **mét**)

---

## **Dữ Liệu Đầu Ra (Output):**

**Thêm 2 cột vào h3_gdf:**

```python
h3_gdf['distance_m']   # Khoảng cách (mét)
h3_gdf['distance_km']  # Khoảng cách (km) = distance_m / 1000
```

**Ví dụ kết quả:**

| h3_index | geometry | distance_m | distance_km | pct_water (cũ) |
|----------|----------|-----------|------------|-----------------|
| 895a829ffffffff | Polygon | 150.2 | 0.150 | 75% |
| 895a82affffffff | Polygon | 2500.8 | 2.501 | 45% |
| 895a82bffffffff | Polygon | 50000.1 | 50.000 | 15% |

---

## **Ý Nghĩa Kết Quả:**

| Khoảng Cách | Phân Loại | Ý Nghĩa |
|------------|----------|---------|
| < 500 m | **Very Close** | Ven biển, cửa sông, cảng |
| 500m - 5 km | **Close** | Khu vực ven biển lân cận |
| 5 - 20 km | **Medium** | Vùng trung gian |
| > 20 km | **Far** | Nội địa sâu |

---

## **Code Cơ Bản (Pseudocode):**

```python
# 1. Chuyển CRS
h3_utm = h3_gdf.to_crs('EPSG:32648')
roi_utm = roi_gdf.to_crs('EPSG:32648')

# 2. Gộp ranh giới
boundary = roi_utm.geometry.unary_union

# 3. Tính centroid + distance
h3_gdf['distance_m'] = h3_utm.geometry.apply(
    lambda geom: geom.centroid.distance(boundary)
)

# 4. Chuyển km
h3_gdf['distance_km'] = h3_gdf['distance_m'] / 1000
```

---

## **Đặc Điểm Quan Trọng:**

✅ **Không dùng bất kỳ raster nào** → Chỉ dùng geometry vector  
✅ **Không phụ thuộc LandCover hay pct_water** → Độc lập hoàn toàn  
✅ **Nhanh & hiệu quả** → Tính hình học thuần túy  
✅ **Có thể tái sử dụng** → Kết hợp với các feature khác sau

In [ ]:
# Debug: Kiểm tra extent của ROI và H3 grid
print("🔍 KIỂM TRA EXTENT:\n")

# Extent ROI
roi_utm = roi_data.to_crs('EPSG:32648')
roi_bounds = roi_utm.total_bounds
roi_width = (roi_bounds[2] - roi_bounds[0]) / 1000  # km
roi_height = (roi_bounds[3] - roi_bounds[1]) / 1000  # km
print(f"📍 ROI DBSCL Extent:")
print(f"   Chiều rộng (E-W): {roi_width:.1f} km")
print(f"   Chiều cao (N-S):  {roi_height:.1f} km")

# Extent H3
h3_utm = h3_data.to_crs('EPSG:32648')
h3_bounds = h3_utm.total_bounds
h3_width = (h3_bounds[2] - h3_bounds[0]) / 1000  # km
h3_height = (h3_bounds[3] - h3_bounds[1]) / 1000  # km
print(f"\n📊 H3 Grid Extent:")
print(f"   Chiều rộng (E-W): {h3_width:.1f} km")
print(f"   Chiều cao (N-S):  {h3_height:.1f} km")
print(f"   Số ô H3: {len(h3_data)}")

# H3 resolution
print(f"\n🔧 H3 Resolution được dùng là bao nhiêu?")
print(f"   (Kiểm tra ở cell 24 - tham số resolution)")

# Kiểm tra cell nào có distance max
max_idx = h3_data['distance_km'].idxmax()
max_cell = h3_data.loc[max_idx]
print(f"\n⭐ Cell có distance max:")
print(f"   Distance: {max_cell['distance_km']:.3f} km")
print(f"   Geometry: {max_cell.geometry}")
